11-1: Suppose one of your co-workers is expecting a baby and you are participing in an office pool to predict the date of birth. assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to to be available to the people in the pool. 

In [4]:
import pandas as pd
import thinkstats2
import thinkplot
import sys
import math
import numpy as np
import nsfg
import random
import hypothesis
import statsmodels.formula.api as smf
import re

In [ ]:
resp = nsfg.ReadFemResp()
live, firsts, others = first.MakeFrames()
live = live[live.prglngth > 30]
resp.index = resp.caseid
join = live.join(resp, on = "caseid", rsuffix = '_r')

In [8]:
#variables used are age at pregnancy, the baby's sex and if this is the first baby or not. 
formula = ('prglngth ~ agepreg + babysex + birthord')
results = smf.ols(formula, data=join).fit()
results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     4.485
Date:                Mon, 06 Feb 2023   Prob (F-statistic):            0.00376
Time:                        14:55:49   Log-Likelihood:                -18281.
No. Observations:                8878   AIC:                         3.657e+04
Df Residuals:                    8874   BIC:                         3.660e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.0174      0.111    352.290      0.000      38.800      39.234
agepreg        0.0027      0.004      0.714      0.475      -0.005       0.010
babysex       -0.0534      0.040     -1.325      0.185      -0.132       0.026
birthord      -0.0699      0.021     -3.402      0.001      -0.110      -0.030
==============================================================================
Omnibus:                     1604.998   Durbin-Watson:                   1.629
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6164.225
Skew:                          -0.865   Prob(JB):                         0.00
Kurtosis:                       6.698   Cond. No.                         144.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

11-3: If the quantity you want to predict is a count, you can use poisson regression, which is implemented in StatsModels with a function called poisson. it works the same way as ols and logit. As an exercise, let's use it to predict how many children a woman has born; in the NSFG dataset, this variable is called numbabes. Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children woould you predict she has born? 

In [10]:
#factors that could affect how many babies someone has and their values
formula = ('numbabes ~ C(race) + educat + age_r + totinc')
results = smf.poisson(formula, data = join).fit()
results.summary()


Optimization terminated successfully.
         Current function value: 1.691999
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Mon, 06 Feb 2023   Pseudo R-squ.:                 0.02825
Time:                        15:31:14   Log-Likelihood:                -15032.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.132e-186
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.1569      0.045     25.968      0.000       1.070       1.244
C(race)[T.2]    -0.1695      0.014    -11.746      0.000      -0.198      -0.141
C(race)[T.3]    -0.1053      0.025     -4.293      0.000      -0.153      -0.057
educat          -0.0561      0.003    -21.182      0.000      -0.061      -0.051
age_r            0.0193      0.001     19.227      0.000       0.017       0.021
totinc          -0.0003      0.000     -1.042      0.297      -0.001       0.000
================================================================================
"""

totincr


In [16]:
#prediction model
columns = ['age_r','race', 'totinc', 'educat']
new = pd.DataFrame([[35, 1, 14, 16]], columns = columns)
y = results.predict(new)
y
#about 2.5 children!!! 

0    2.539338
dtype: float64

11-4: If the quantity you want to rpedict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called mnlogit. As an exercise, let's use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; int he NSFG dataset, marital status is enconded in a variable called rmarital. Suppose you meet a woman who is 25 years old, white, adn a high school graduate whose annual housebold income is about $45,000. What is the probability that she is married, cohabitating, etc? 

In [22]:
formula = ('rmarital ~ C(race) + educat + age_r + totinc')
results = smf.mnlogit(formula, data = join).fit()
results.summary()


Optimization terminated successfully.
         Current function value: 1.152503
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Mon, 06 Feb 2023   Pseudo R-squ.:                  0.1157
Time:                        16:28:09   Log-Likelihood:                -10239.
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.7896      0.279     17.191      0.000       4.244       5.336
C(race)[T.2]    -1.0355      0.088    -11.823      0.000      -1.207      -0.864
C(race)[T.3]    -0.6363      0.134     -4.739      0.000      -0.899      -0.373
educat          -0.2858      0.018    -16.329      0.000      -0.320      -0.252
age_r           -0.0662      0.006    -11.468      0.000      -0.077      -0.055
totinc          -0.0061      0.002     -3.427      0.001      -0.010      -0.003
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.9922      0.907     -4.403      0.000      -5.769      -2.215
C(race)[T.2]    -0.6376      0.235     -2.713      0.007      -1.098      -0.177
C(race)[T.3]     0.1105      0.332      0.332      0.740      -0.541       0.762
educat          -0.2871      0.045     -6.428      0.000      -0.375      -0.200
age_r            0.1099      0.019      5.800      0.000       0.073       0.147
totinc           0.0041      0.003      1.240      0.215      -0.002       0.011
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.0452      0.294     -6.965      0.000      -2.621      -1.470
C(race)[T.2]    -0.5078      0.091     -5.604      0.000      -0.685      -0.330
C(race)[T.3]    -0.7637      0.167     -4.560      0.000      -1.092      -0.435
educat          -0.0717      0.015     -4.865      0.000      -0.101      -0.043
age_r            0.0495      0.006      7.743      0.000       0.037       0.062
totinc          -0.0091      0.002     -4.250      0.000      -0.013      -0.005
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.5688      0.333      4.713      0.000       0.916       2.221
C(race)[T.2]    -1.2782      0.100    -12.763      0.000      -1.475      -1.082
C(race)[T.3]    -0.5778      0.151     -3.835      0.000      -0.873      -0.283
educat          -0.1849      0.019     -9.517      0.000      -0.223      -0.147
age_r           -0.0117      0.007     -1.660      0.097      -0.026       0.002
totinc          -0.0117      0.003     -4.433      0.000      -0.017      -0.007
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.8653      0.263     26.142      0.000       6.351       7.380
C

In [23]:
columns = ['age_r','race', 'totinc', 'educat']
new = pd.DataFrame([[25, 2, 11, 12]], columns = columns)
y = results.predict(new)
y

,0,1,2,3,4,5
0,0.581084,0.143774,0.002954,0.05965,0.05546,0.157078


In [ ]:
#my 11-3 matches but not my 11-4. also, what is r2??? 